In [35]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "Biology24"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [36]:
import pandas as pd

# Load your CSV
df = pd.read_csv('/Users/jaeeponde/dsm_project/st_louis/cleaned/st_louis_restaurants.csv')
df

,business_id,name,city,state,stars,review_count,attributes,categories
0,yM8LlTInbQH4FwWC97lz6w,McDonald's,St. Louis,MO,1.5,100,"{'Alcohol': ""u'none'"", 'BikeParking': 'True', ...","Fast Food, Restaurants, Food, Burgers, Coffee ..."
1,hsyIDX4x4ndYEvPoJF6_1g,Merendero La Catrachitas,St. Louis,MO,4.5,22,"{'RestaurantsTableService': 'True', 'Caters': ...","Latin American, Honduran, Restaurants"
2,9qxMhGWjGsuPUoQB3Y74Wg,McDonald's,St. Louis,MO,1.5,62,"{'BikeParking': 'True', 'RestaurantsAttire': ""...","Food, Restaurants, Coffee & Tea, Burgers, Fast..."
3,wMyHVb54Ne9SNasa4kjOCg,Carnivore,St. Louis,MO,4.5,281,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Restaurants, Breakfast & Brunch, Steakhouses, ..."
4,MuKoTR56s6elHEI2wUkgJA,Grace Meat + Three,St. Louis,MO,4.5,470,"{'WiFi': ""u'no'"", 'HasTV': 'True', 'Restaurant...","Restaurants, Soul Food, Southern, Soup, Sandwi..."
...,...,...,...,...,...,...,...,...
538,8RuoB-7s1mlOX4oJwLJ9Yg,Island Life Yoga & Nutrition,St. Louis,MO,4.5,8,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Yoga, Active Life, Food, Specialty Food, Fitne..."
539,1hUdXlgGBkuLevaPHTGeHA,Baba Xavi,St. Louis,MO,5.0,22,"{'Ambience': ""{'touristy': False, 'hipster': F...","Restaurants, Indian"
540,sBTUg4DINvmrfM2cLQZAjw,Hook & Reel Cajun Seafood & Bar,St. Louis,MO,3.5,87,"{'RestaurantsPriceRange2': '2', 'GoodForKids':...","Cajun/Creole, Restaurants, Seafood"
541,hmp38bs96zxjyNOnMAsG9g,Fleming’s Prime Steakhouse & Wine Bar,St. Louis,MO,4.5,259,"{'HasTV': 'True', 'BusinessAcceptsCreditCards'...","Steakhouses, Nightlife, Seafood, Restaurants, ..."


In [37]:
# Function to create business nodes
def create_business(tx, row):
    query = """
    MERGE (b:Business {business_id: $business_id})
    SET b.name = $name,
        b.city = $city,
        b.state = $state,
        b.stars = $stars,
        b.review_count = $review_count,
        b.attributes = $attributes,
        b.categories = $categories
    """
    tx.run(query,
           business_id=row['business_id'],
           name=row['name'],
           city=row['city'],
           state=row['state'],
           stars=float(row['stars']),
           review_count=int(row['review_count']),
           attributes=str(row['attributes']),
           categories=str(row['categories']))

# Ensure that you are targeting the "Philadelphia" database (change this to the correct name if needed)
with driver.session(database="neo4j") as session:  # Specify database here
    for _, row in df.iterrows():
        session.write_transaction(create_business, row)

driver.close()
print("Business nodes successfully created in the Philadelphia database.")


/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/595590798.py:26: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_business, row)


Business nodes successfully created in the Philadelphia database.


In [38]:
from neo4j import GraphDatabase
import pandas as pd

# Load filtered user data
user_core = pd.read_csv("/Users/jaeeponde/dsm_project/st_louis/cleaned/st_louis_user_core.csv")

# Function to upload user data
def upload_users_to_neo4j():
    with driver.session(database="neo4j") as session:
        for _, row in user_core.iterrows():
            user_id = row['user_id']
            name = row['name']
            review_count = row['review_count']
            average_stars = row['average_stars']

            # Create User node in Neo4j
            session.write_transaction(create_user_node, user_id, name, review_count, average_stars)

# Function to create a User node
def create_user_node(tx, user_id, name, review_count, average_stars):
    query = (
        "MERGE (u:User {user_id: $user_id}) "
        "SET u.name = $name, u.review_count = $review_count, u.average_stars = $average_stars"
    )
    tx.run(query, user_id=user_id, name=name, review_count=review_count, average_stars=average_stars)

# Upload users to Neo4j
upload_users_to_neo4j()

# Close Neo4j connection
driver.close()

print(f"Uploaded {len(user_core)} users to Neo4j.")


/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/3154023456.py:9: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:
/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/3154023456.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_user_node, user_id, name, review_count, average_stars)


Uploaded 1757 users to Neo4j.


In [39]:
import pandas as pd
from neo4j import GraphDatabase

# Load user friendship data (user pairs)
friend_edges = pd.read_csv("/Users/jaeeponde/dsm_project/st_louis/cleaned/st_louis_user_edges.csv")

# Function to upload user friendships to Neo4j
def upload_user_edges_to_neo4j():
    with driver.session(database="neo4j") as session:
        for _, row in friend_edges.iterrows():
            user_id_1 = row['user_id_1']
            user_id_2 = row['user_id_2']

            # Create FRIEND relationship between user 1 and user 2
            session.write_transaction(create_friendship_edge, user_id_1, user_id_2)

# Function to create a FRIEND relationship between users
def create_friendship_edge(tx, user_id_1, user_id_2):
    query = (
        "MATCH (u1:User {user_id: $user_id_1}), (u2:User {user_id: $user_id_2}) "
        "MERGE (u1)-[:FRIEND]->(u2) "
        "MERGE (u2)-[:FRIEND]->(u1)"  # To ensure bidirectional friendship
    )
    tx.run(query, user_id_1=user_id_1, user_id_2=user_id_2)

# Upload user friendships to Neo4j
upload_user_edges_to_neo4j()

# Close Neo4j connection
driver.close()

print(f"Uploaded {len(friend_edges)} user friendships (edges) to Neo4j.")


/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/1492863078.py:9: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:
/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/1492863078.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_friendship_edge, user_id_1, user_id_2)


Uploaded 6974 user friendships (edges) to Neo4j.


In [40]:
import pandas as pd
from tqdm import tqdm

# Load the review data
file_path = "/Users/jaeeponde/dsm_project/st_louis/cleaned/st_louis_reviews.csv"  # Update with actual path
df = pd.read_csv(file_path)

# Clean the data
df = df.dropna(subset=['review_id', 'user_id', 'business_id', 'stars', 'text'])
df = df[~df['business_id'].astype(str).str.contains("#NAME")]
df = df[~df['date'].astype(str).str.contains("#")]

# Function to create Review nodes and connect them to User and Business
def create_review(tx, row):
    query = """
    MERGE (u:User {user_id: $user_id})
    MERGE (b:Business {business_id: $business_id})
    CREATE (r:Review {
        review_id: $review_id,
        stars: $stars,
        text: $text,
        date: $date
    })
    MERGE (u)-[:WROTE]->(r)
    MERGE (r)-[:REVIEWS]->(b)
    """
    tx.run(query,
           review_id=row['review_id'],
           user_id=row['user_id'],
           business_id=row['business_id'],
           stars=int(row['stars']),
           text=str(row['text']),
           date=str(row['date']))

# Insert into Neo4j
with driver.session(database="neo4j") as session:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Uploading reviews"):
        session.write_transaction(create_review, row)

driver.close()
print("Reviews successfully added and connected to Users and Businesses.")

/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/3761042905.py:36: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:
Uploading reviews:   0%|          | 0/4817 [00:00<?, ?it/s]/var/folders/29/458ydd_j11q4vpw8md8c37fh0000gn/T/ipykernel_12174/3761042905.py:38: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_review, row)
Uploading reviews: 100%|██████████| 4817/4817 [00:42<00:00, 112.54it/s]

Reviews successfully added and connected to Users and Businesses.
